In [1]:
Nbr_Files_to_Download = 2300
ws ='python'
lh ='data'
table_path            = f'abfss://{ws}@onelake.dfs.fabric.microsoft.com/{lh}.Lakehouse/Tables/aemo/'

In [2]:
!pip install duckdb     --upgrade
!pip install deltalake  --upgrade
!pip install -q boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 61.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.0.0
    Uninstalling duckdb-1.0.0:
      Successfully uninstalled duckdb-1.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 22.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: deltalake
    Found existing installation: deltalake 0.18.2
    Uninstalling deltalake-0.18.2:
      Successfully uninstalled deltalake-0.18.2


In [3]:
from   deltalake.writer import try_get_deltatable , write_deltalake
from   deltalake import DeltaTable
import boto3
import glob
import duckdb
from   datetime import datetime
import time
import os
from   psutil import *
import re
import requests
from   shutil import unpack_archive
from   urllib.request import urlopen
import multiprocessing

In [4]:
vCPU = str(cpu_count()) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = vCPU+' '+str(mem)+'GB'
runtime

'2 vCPU 15.0GB'

In [5]:
def get_s3_bucket_session():
    #### use azure key vault, otherwise store data in myworkspace and don't share it with anyone
    # input the abfss path of ini file, does not require a mounted path
    duckdb.sql(f""" CREATE or replace SECRET onelake (TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{notebookutils.credentials.getToken('storage')}') """)
    kv =duckdb.sql(""" select trim(split_part("[myvars]",' : ', 1) ) as var, trim(split_part("[myvars]",' : ', 2)) as values from
    read_csv('abfss://python@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/variable.ini')
        """)
    kv_values=duckdb.sql( " from kv where var in ('region_name' ,'endpoint_url','aws_access_key_id','aws_secret_access_key') ").fetchall()
    for key, value in kv_values:
        globals()[key] = value

    s3_resource = boto3.resource('s3',
            region_name            =  region_name,
            endpoint_url           =  endpoint_url ,
            aws_access_key_id      =  aws_access_key_id ,
            aws_secret_access_key  =  aws_secret_access_key

                )
    return s3_resource

# Ingest Data

**<mark>Download Some Data from the web</mark>**

In [6]:
def download(url,Path,total_files):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique, reverse=True)
    current =  [os.path.basename(x) for x in glob.glob(Path+'*.zip')]
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))[:total_files]
    print(str(len(files_to_upload)) + ' New File Downloaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

In [7]:
def uncompress(x):
        unpack_archive(str(Zip_Path+x), str(uncompressed_Path), 'zip')
def unzip(Source, Destination):
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True)
    filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    ### checl the unzipped files already
    current_csv = [os.path.basename(x) for x in glob.glob(Destination+'*.CSV')]
    current_json = [os.path.basename(x) for x in glob.glob(Destination+'*.json')]
    current = current_csv + current_json
    current = [w.replace('.CSV','.zip') for w in current]
    current = [w.replace('.json','.zip') for w in current]
    current = [w.replace('SCADA','FacilityScada') for w in current]
    current = [w.replace('-', '') for w in current]
    #unzip only the delta
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))
    print(str(len(files_to_upload)) + ' New File uncompressed')
    if len(files_to_upload) != 0 :
      with multiprocessing.Pool() as pool:
       for _ in pool.imap_unordered(uncompress, files_to_upload, chunksize=1):
         pass
      return "done"
    else:
     return "nothing to see here"

In [8]:
def syncremotestorage(bucketName, remoteDirectoryName,Nbr_Files_to_Download):
    location = "/lakehouse/default/Files/0_Source/"
    bucket = s3_resource.Bucket(bucketName)
    ############# zip on disk
    files_disk=[os.path.basename(x) for x in glob.glob(location+remoteDirectoryName+'/*.zip')]
    files_disk = [remoteDirectoryName+'/' + i for i in files_disk]
    ################# zip on remote storage
    files_R2 = bucket.objects.filter(Prefix = remoteDirectoryName)
    files_R2 = [obj.key for obj in sorted(files_R2, key=lambda x: x.last_modified, reverse=True)]
    ##################################
    files_to_download = list( set(files_R2)- set(files_disk))
    files_to_download = list(dict.fromkeys(files_to_download))
    files_to_download=sorted(files_to_download, reverse=True)
    for obj in files_to_download[:Nbr_Files_to_Download]:
        print('download '+obj)
        if not os.path.exists(os.path.dirname(obj)):
           os.makedirs(os.path.dirname(obj))
        bucket.download_file(obj, location+obj)
    ########################################
    files_to_upload = list(  set(files_disk) - set(files_R2))
    files_to_upload = list(dict.fromkeys(files_to_upload))
    for file in files_to_upload:
        print('upload '+file)
        bucket.upload_file(location+file, file)


**<mark>Check files already Ingested</mark>**

In [9]:
def get_Path(Source,Destination):
 dt =try_get_deltatable(Destination,storage_options={"allow_unsafe_rename":"true"})
 if dt is not None:
    existing_files = duckdb.sql(f""" select distinct file as file from delta_scan('{Destination}') """).df()['file'].tolist()
 else:
  existing_files=[]
 print(len(existing_files))
 list_files_csv=[os.path.basename(x) for x in glob.glob(Source+'*.CSV')]
 list_files_json=[os.path.basename(x) for x in glob.glob(Source+'*.json')]
 filelist = list_files_csv + list_files_json

 files_to_upload = list(set(filelist) - set(existing_files))
 files_to_upload = list(dict.fromkeys(files_to_upload))
 files_to_upload_full_Path = [Source + i for i in files_to_upload]
 return files_to_upload_full_Path[:Nbr_Files_to_Download]

**<mark>scada </mark>**

In [10]:
def get_scada(files_to_upload_full_Path):
    raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
    Skip=1,header =0,all_varchar=1,
    columns={{
    'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
    'DUID': 'VARCHAR','INTERVENTION': 'VARCHAR','DISPATCHMODE': 'VARCHAR','AGCSTATUS': 'VARCHAR','INITIALMW': 'VARCHAR',
    'TOTALCLEARED': 'VARCHAR','RAMPDOWNRATE': 'VARCHAR','RAMPUPRATE': 'VARCHAR','LOWER5MIN': 'VARCHAR',
    'LOWER60SEC': 'VARCHAR','LOWER6SEC': 'VARCHAR','RAISE5MIN': 'VARCHAR','RAISE60SEC': 'VARCHAR',
    'RAISE6SEC': 'VARCHAR','MARGINAL5MINVALUE': 'VARCHAR','MARGINAL60SECVALUE': 'VARCHAR',
    'MARGINAL6SECVALUE': 'VARCHAR','MARGINALVALUE': 'VARCHAR','VIOLATION5MINDEGREE': 'VARCHAR',
    'VIOLATION60SECDEGREE': 'VARCHAR','VIOLATION6SECDEGREE': 'VARCHAR','VIOLATIONDEGREE': 'VARCHAR',
    'LOWERREG': 'VARCHAR','RAISEREG': 'VARCHAR','AVAILABILITY': 'VARCHAR','RAISE6SECFLAGS': 'VARCHAR',
    'RAISE60SECFLAGS': 'VARCHAR','RAISE5MINFLAGS': 'VARCHAR','RAISEREGFLAGS': 'VARCHAR',
    'LOWER6SECFLAGS': 'VARCHAR','LOWER60SECFLAGS': 'VARCHAR','LOWER5MINFLAGS': 'VARCHAR',
    'LOWERREGFLAGS': 'VARCHAR','RAISEREGAVAILABILITY': 'VARCHAR','RAISEREGENABLEMENTMAX': 'VARCHAR',
    'RAISEREGENABLEMENTMIN': 'VARCHAR','LOWERREGAVAILABILITY': 'VARCHAR','LOWERREGENABLEMENTMAX': 'VARCHAR',
    'LOWERREGENABLEMENTMIN': 'VARCHAR','RAISE6SECACTUALAVAILABILITY': 'VARCHAR',
    'RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
    'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR',
    'LOWER60SECACTUALAVAILABILITY': 'VARCHAR','LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR'
    }},
    filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
    where I='D' and UNIT ='DUNIT' AND VERSION = 3                  """)

    final_scada=duckdb.sql("""
    select
    UNIT,
    DUID,
    cast(columns(*exclude(DUID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	  isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
    return final_scada.record_batch()


**<mark>Price </mark>**

In [11]:
def get_price(files_to_upload_full_Path):
  raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
  Skip=1,header =0,all_varchar=1,
  columns={{
  'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
  'REGIONID': 'VARCHAR','INTERVENTION': 'VARCHAR','RRP': 'VARCHAR','EEP': 'VARCHAR','ROP': 'VARCHAR','APCFLAG': 'VARCHAR',
  'MARKETSUSPENDEDFLAG': 'VARCHAR','TOTALDEMAND': 'VARCHAR','DEMANDFORECAST': 'VARCHAR','DISPATCHABLEGENERATION': 'VARCHAR',
  'DISPATCHABLELOAD': 'VARCHAR','NETINTERCHANGE': 'VARCHAR','EXCESSGENERATION': 'VARCHAR','LOWER5MINDISPATCH': 'VARCHAR',
  'LOWER5MINIMPORT': 'VARCHAR','LOWER5MINLOCALDISPATCH': 'VARCHAR','LOWER5MINLOCALPRICE': 'VARCHAR','LOWER5MINLOCALREQ': 'VARCHAR',
  'LOWER5MINPRICE': 'VARCHAR','LOWER5MINREQ': 'VARCHAR','LOWER5MINSUPPLYPRICE': 'VARCHAR','LOWER60SECDISPATCH': 'VARCHAR','LOWER60SECIMPORT': 'VARCHAR',
  'LOWER60SECLOCALDISPATCH': 'VARCHAR','LOWER60SECLOCALPRICE': 'VARCHAR','LOWER60SECLOCALREQ': 'VARCHAR','LOWER60SECPRICE': 'VARCHAR',
  'LOWER60SECREQ': 'VARCHAR','LOWER60SECSUPPLYPRICE': 'VARCHAR','LOWER6SECDISPATCH': 'VARCHAR','LOWER6SECIMPORT': 'VARCHAR',
  'LOWER6SECLOCALDISPATCH': 'VARCHAR','LOWER6SECLOCALPRICE': 'VARCHAR','LOWER6SECLOCALREQ': 'VARCHAR','LOWER6SECPRICE': 'VARCHAR',
  'LOWER6SECREQ': 'VARCHAR','LOWER6SECSUPPLYPRICE': 'VARCHAR','RAISE5MINDISPATCH': 'VARCHAR','RAISE5MINIMPORT': 'VARCHAR',
  'RAISE5MINLOCALDISPATCH': 'VARCHAR','RAISE5MINLOCALPRICE': 'VARCHAR','RAISE5MINLOCALREQ': 'VARCHAR','RAISE5MINPRICE': 'VARCHAR',
  'RAISE5MINREQ': 'VARCHAR','RAISE5MINSUPPLYPRICE': 'VARCHAR','RAISE60SECDISPATCH': 'VARCHAR','RAISE60SECIMPORT': 'VARCHAR',
  'RAISE60SECLOCALDISPATCH': 'VARCHAR','RAISE60SECLOCALPRICE': 'VARCHAR','RAISE60SECLOCALREQ': 'VARCHAR','RAISE60SECPRICE': 'VARCHAR',
  'RAISE60SECREQ': 'VARCHAR','RAISE60SECSUPPLYPRICE': 'VARCHAR','RAISE6SECDISPATCH': 'VARCHAR','RAISE6SECIMPORT': 'VARCHAR',
  'RAISE6SECLOCALDISPATCH': 'VARCHAR','RAISE6SECLOCALPRICE': 'VARCHAR','RAISE6SECLOCALREQ': 'VARCHAR','RAISE6SECPRICE': 'VARCHAR',
  'RAISE6SECREQ': 'VARCHAR','RAISE6SECSUPPLYPRICE': 'VARCHAR','AGGREGATEDISPATCHERROR': 'VARCHAR','AVAILABLEGENERATION': 'VARCHAR',
  'AVAILABLELOAD': 'VARCHAR','INITIALSUPPLY': 'VARCHAR','CLEAREDSUPPLY': 'VARCHAR','LOWERREGIMPORT': 'VARCHAR','LOWERREGLOCALDISPATCH': 'VARCHAR',
  'LOWERREGLOCALREQ': 'VARCHAR','LOWERREGREQ': 'VARCHAR','RAISEREGIMPORT': 'VARCHAR','RAISEREGLOCALDISPATCH': 'VARCHAR','RAISEREGLOCALREQ': 'VARCHAR',
  'RAISEREGREQ': 'VARCHAR','RAISE5MINLOCALVIOLATION': 'VARCHAR','RAISEREGLOCALVIOLATION': 'VARCHAR','RAISE60SECLOCALVIOLATION': 'VARCHAR',
  'RAISE6SECLOCALVIOLATION': 'VARCHAR','LOWER5MINLOCALVIOLATION': 'VARCHAR','LOWERREGLOCALVIOLATION': 'VARCHAR','LOWER60SECLOCALVIOLATION': 'VARCHAR',
  'LOWER6SECLOCALVIOLATION': 'VARCHAR','RAISE5MINVIOLATION': 'VARCHAR','RAISEREGVIOLATION': 'VARCHAR','RAISE60SECVIOLATION': 'VARCHAR',
  'RAISE6SECVIOLATION': 'VARCHAR','LOWER5MINVIOLATION': 'VARCHAR','LOWERREGVIOLATION': 'VARCHAR','LOWER60SECVIOLATION': 'VARCHAR',
  'LOWER6SECVIOLATION': 'VARCHAR','RAISE6SECRRP': 'VARCHAR','RAISE6SECROP': 'VARCHAR','RAISE6SECAPCFLAG': 'VARCHAR','RAISE60SECRRP': 'VARCHAR',
  'RAISE60SECROP': 'VARCHAR','RAISE60SECAPCFLAG': 'VARCHAR','RAISE5MINRRP': 'VARCHAR','RAISE5MINROP': 'VARCHAR','RAISE5MINAPCFLAG': 'VARCHAR',
  'RAISEREGRRP': 'VARCHAR','RAISEREGROP': 'VARCHAR','RAISEREGAPCFLAG': 'VARCHAR','LOWER6SECRRP': 'VARCHAR','LOWER6SECROP': 'VARCHAR',
  'LOWER6SECAPCFLAG': 'VARCHAR','LOWER60SECRRP': 'VARCHAR','LOWER60SECROP': 'VARCHAR','LOWER60SECAPCFLAG': 'VARCHAR','LOWER5MINRRP': 'VARCHAR',
  'LOWER5MINROP': 'VARCHAR','LOWER5MINAPCFLAG': 'VARCHAR','LOWERREGRRP': 'VARCHAR','LOWERREGROP': 'VARCHAR','LOWERREGAPCFLAG': 'VARCHAR',
  'RAISE6SECACTUALAVAILABILITY': 'VARCHAR','RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
  'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR','LOWER60SECACTUALAVAILABILITY': 'VARCHAR',
  'LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR','LORSURPLUS': 'VARCHAR','LRCSURPLUS': 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and UNIT ='DREGION' AND VERSION = 3
                  """)
  final_price=duckdb.sql("""
    select
    UNIT,
    REGIONID,
    cast(columns(*exclude(REGIONID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	  isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
  return final_price.record_batch()

**<mark>WA </mark>**

In [12]:
def get_wa(files_to_upload_full_Path):
    final_wa=duckdb.sql(f"""
        select data.code as DUID,
        cast(null as varchar) as UNIT,
        cast(null as double) as LOWERREGENABLEMENTMAX ,
        cast(null as double) as MARGINALVALUE,
        data.quantity as INITIALMW,
        cast(null as double) as RAISEREGACTUALAVAILABILITY,
        cast(null as double) as RAISEREGAVAILABILITY ,
        cast(null as double) as LOWER6SECACTUALAVAILABILITY ,
        cast(null as double) as RUNNO ,
        cast(null as double) as LOWER5MIN ,
        cast(null as double) as AGCSTATUS ,
        cast(0 as double) as INTERVENTION ,
        cast(null as double) as LOWER6SECFLAGS ,
        cast(null as double) as MARGINAL6SECVALUE ,
        cast(null as double) as RAISE60SECACTUALAVAILABILITY ,
        cast(null as double) as RAISE5MIN ,
        cast(null as double) as  AVAILABILITY,
        cast(null as double) as RAISEREGFLAGS ,
        cast(null as double) as VIOLATION6SECDEGREE ,
        cast(null as double) as LOWERREGAVAILABILITY ,
        cast(null as double) as TOTALCLEARED ,
        cast(null as double) as LOWER60SECFLAGS ,
        cast(null as double) as LOWERREGENABLEMENTMIN ,
        cast(null as double) as LOWERREGACTUALAVAILABILITY ,
        cast(null as double) as RAISEREGENABLEMENTMIN ,
        cast(null as double) as RAMPUPRATE ,
        cast(null as double) as VIOLATIONDEGREE ,
        cast(null as double) as LOWER5MINFLAGS ,
        cast(null as double) as RAISE6SEC ,
        cast(null as double) as LOWER60SECACTUALAVAILABILITY ,
        cast(null as double) as MARGINAL5MINVALUE ,
        cast(null as double) as RAMPDOWNRATE ,
        cast(null as double) as DISPATCHMODE ,
        cast(null as double) as RAISE5MINACTUALAVAILABILITY ,
        cast(null as double) as VIOLATION60SECDEGREE ,
        cast(null as double) as RAISE60SECFLAGS ,
        cast(null as double) as RAISEREGENABLEMENTMAX ,
        cast(null as double) as RAISE6SECFLAGS ,
        cast(null as double) as RAISE60SEC ,
        cast(null as double) as MARGINAL60SECVALUE ,
        cast(null as double) as LOWER60SEC ,
        cast(null as double) as RAISE5MINFLAGS ,
        cast(null as double) as LOWERREGFLAGS ,
        cast(null as double) as LOWER5MINACTUALAVAILABILITY ,
        cast(null as double) as VIOLATION5MINDEGREE ,
        cast(null as double) as VERSION ,
        cast(null as double) as RAISE6SECACTUALAVAILABILITY ,
        cast(null as double) as RAISEREG ,
        cast(null as double) as LOWERREG ,
        cast(null as double) as LOWER6SEC ,
        cast(data.dispatchInterval AS TIMESTAMPTZ) SETTLEMENTDATE  ,
        cast(SETTLEMENTDATE as date) as date,
        0 as PRIORITY ,
        parse_filename(filename) as file,
        isoyear (cast (data.dispatchInterval as timestamp)) as YEAR
        from (select unnest(data.facilityScadaDispatchIntervals) as data,filename
        from read_json_auto({files_to_upload_full_Path},filename=1)) """)
########
    df_wa=final_wa.record_batch()
    return df_wa

# Process Data

In [13]:
Web_Path                = "https://nemweb.com.au/Reports/Current/Daily_Reports/"
Zip_Path                = "/lakehouse/default/Files/0_Source/aemo/"
uncompressed_Path       = "/lakehouse/default/Files/1_Transform/CSV/Daily_Reports/"
#############################################
download(Web_Path,Zip_Path,Nbr_Files_to_Download)
#### Optionally store raw data in remote storage
try:
  s3_resource=get_s3_bucket_session()
  syncremotestorage("archive","aemo",Nbr_Files_to_Download)
except:
  print('wrong credential')
unzip(Zip_Path,uncompressed_Path)

0 New File Downloaded
0 New File uncompressed


'nothing to see here'

In [14]:
%%time
Onelake_table           = table_path+'price'
files_to_upload_full_Path = get_Path(uncompressed_Path,Onelake_table)
if len(files_to_upload_full_Path) >0 :
  df = get_price(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",engine='pyarrow')
else:
  print('all loaded already')


2445
all loaded already
CPU times: user 1.21 s, sys: 112 ms, total: 1.32 s
Wall time: 3.86 s


In [15]:
%%time
Onelake_table           = table_path+'scada'
files_to_upload_full_Path = get_Path(uncompressed_Path,Onelake_table)
if len(files_to_upload_full_Path) >0 :
    df = get_scada(files_to_upload_full_Path)
    write_deltalake(Onelake_table, df,mode="append",partition_by=['YEAR'],engine='pyarrow')
    del df
else:
  print('all loaded already')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2880
all loaded already
CPU times: user 11.1 s, sys: 263 ms, total: 11.4 s
Wall time: 1min 5s


In [16]:
%%time
Web_Path           = "https://data.wa.aemo.com.au/public/market-data/wemde/facilityScada/previous/"
Zip_Path           = "/lakehouse/default/Files/0_Source/wa/"
uncompressed_Path  = "/lakehouse/default/Files/1_Transform/JSON/WA/facilityScada/"
Onelake_table      = table_path+'scada'

download(Web_Path,Zip_Path,Nbr_Files_to_Download)
#### Optionally store raw data in remote storage
try:
  s3_resource=get_s3_bucket_session()
  syncremotestorage("archive","wa",Nbr_Files_to_Download)
except:
  print('wrong credential')
unzip(Zip_Path,uncompressed_Path)
files_to_upload_full_Path=get_Path(uncompressed_Path,Onelake_table)


if len(files_to_upload_full_Path) >0 :
  df = get_wa(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",partition_by=['YEAR'],engine='pyarrow')
  del df
else:
  print('all loaded already')

1 New File Downloaded
upload wa/FacilityScada_20241210.zip
5 New File uncompressed


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

<timed exec>:19: DeprecationWarning: pyarrow engine is deprecated and will be removed in v1.0


CPU times: user 11.6 s, sys: 238 ms, total: 11.9 s
Wall time: 1min 9s
